In [ ]:
import json
import utilities
from tqdm.notebook import tqdm

In [ ]:
base_directory = 'khaleesi/data/'
json_representation_http_dir = base_directory + 'crawl-http-labeled.json'
json_representation_js_dir = base_directory + 'crawl-js-connected-labeled.json'

In [ ]:
results_csv_dir = base_directory + 'detections.csv'

In [ ]:
json_representation_http = utilities.read_json(json_representation_http_dir)
json_representation_js = utilities.read_json(json_representation_js_dir)

In [ ]:
results_csv = set(utilities.read_file_newline_stripped(results_csv_dir))

In [ ]:
results_dict = {}
results_earliest = set()

for item in results_csv:
    key = item.rsplit('|', 1)[0]
    current_value = item.rsplit('|', 1)[1]
    
    if item.startswith('H|'):
        if key in results_dict:
            previous_value = results_dict[key]
            if int(current_value) < int(previous_value):
                results_dict[key] = current_value

        else:
            results_dict[key] = current_value
            
for item in results_dict:
    results_earliest.add(item + '|' + results_dict[item])

for item in results_csv:
    if item.startswith('J|'):
        results_earliest.add(item)

In [ ]:
def compute_accuracy_request(tp,tn,fp,fn,json_representation, detections, detections_dict):
    fl_detected = set()
    our_detected = set()
    pbar = tqdm(total=len(json_representation), position=0, leave=True)

    for key in json_representation:
        pbar.update(1)
        for request in json_representation[key]['content'][1:]:
            redirect_id = request['redirect_id']
            original_label = False 
            detection_label = False
            
            
            if key.startswith('H|') and key in fl_detected:
                original_label = True
            
            if not original_label:
            
                if 'ground_truth' in request:
                    original_label = request['ground_truth']
                    fl_detected.add(key)
                else:
                    original_label = False
            
            if key.startswith('H|') and key in our_detected:
                detection_label = True
            
            if not detection_label:
                if key +'|' + str(redirect_id) in detections:
                    detection_label = True
                    our_detected.add(key)
                elif key.startswith('H|') and key in detections_dict:
                    if int(detections_dict[key]) < int(redirect_id):
                        detection_label = True
                        our_detected.add(key)
                else:
                    detection_label = False

                
            if original_label == True and detection_label == True:
                tp += 1   
            elif original_label == True and detection_label == False:
                fn += 1
            elif original_label == False and detection_label == True:
                fp += 1
            elif original_label == False and detection_label == False:
                tn += 1
            
    return tp,tn,fp,fn

In [ ]:
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0

tp,tn,fp,fn = compute_accuracy_request(tp,tn,fp,fn,json_representation_http, results_earliest, results_dict)
tp,tn,fp,fn = compute_accuracy_request(tp,tn,fp,fn,json_representation_js, results_earliest, results_dict)

In [ ]:
recall = ( (tp) / (tp + fn) ) * 100.0
precision = ( (tp) / (tp + fp) ) * 100.0
fnr = ( (fn) / (fn + tp) ) * 100.0
fpr = ( (fp) / (fp + tn) ) * 100.0
accuracy = ( (tp + tn ) / (tp + tn + fp + fn) ) * 100.0 

In [ ]:
print('Recall:', recall)
print('Precision:', precision)
print('FNR:', fnr)
print('FPR:', fpr)
print('Accuracy:', accuracy)